In [1]:
import os
import pandas as pd
import faiss
import pickle
from sentence_transformers import SentenceTransformer
import openai
import numpy as np


In [2]:
df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")

In [ ]:
documents = []
metadatas = []

for idx, row in df.iterrows():
    chunk = f"'{row['사무명']}'은 {row['사무개요']}"

    documents.append(chunk)
    metadatas.append({
    "사무명": row["사무명"],
    "소관부처": row["소관부처"],
    "담당부서": row["담당부서"],
    "원문": "\n".join([f"[{col}] {row[col]}" for col in df.columns[1:] if pd.notna(row[col])])
})

## 테스트용입니다!
print("청크:\n", documents[0])
print("\n메타데이터:\n", metadatas[0])


청크:
 '토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.

메타데이터:
 {'사무명': '토지임야대장등본교부 다량신청', '소관부처': '국토교통부', '담당부서': '공간정보제도과', '원문': '[사무명] 토지임야대장등본교부 다량신청\n[사무개요] 토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.\n[신청유형] 교부\n[신청자 자격] 누구나신청가능\n[소관부처] 국토교통부\n[담당부서] 공간정보제도과\n[접수방법] 인터넷'}


In [4]:
# # 임베딩 생성
model = SentenceTransformer("EmbeddingModelFinetuning")

In [8]:
embeddings = model.encode(documents, show_progress_bar=True)

# # FAISS 저장
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# # 인덱스와 메타데이터 저장
faiss.write_index(index, "rag_index.faiss")
with open("rag_metadata.pkl", "wb") as f:
    pickle.dump((documents, metadatas), f)

Batches:   0%|          | 0/189 [00:00<?, ?it/s]